In [229]:
%%writefile ../config/patterns.yaml
patterns:
  r1:
    - "_1"
    - "_r1"
    - "_R1"
    - "_r"
    - "_F"
  r2:
    - "_2"
    - "_r2"
    - "_R2"
    - "_f"
    - "_R"
  ignore:
    - "^i_"
    - "^I_"
    - "_i\\d+"
    - "_I\\d+"


Overwriting ../config/patterns.yaml


In [173]:
! ln -s ../config .

In [88]:
%%writefile ../frmatcher/fastq_file_name_checker.py
import os
import re
import yaml
from typing import Dict, List, Optional
from loguru import logger
import importlib.resources as resources  # Modern resource handling for Python 3.7+

class FilenameLengthMismatchError(ValueError):
    """Custom exception for filename length mismatches."""

class PatternsNotLoadedError(ValueError):
    """Custom exception when patterns are not loaded."""

class FastqFileNameChecker:
    def __init__(
        self,
        filenames: List[str],
        config_path: Optional[str] = None,
        length_check: bool = False,
        verbose: bool = False,
    ) -> None:
        """
        Initialize the FastqFileNameChecker with a list of filenames.

        Args:
            filenames (List[str]): List of file paths to categorize.
            config_path (str, optional): Path to the YAML configuration file.
                Defaults to None, and patterns must be manually injected.
            length_check (bool): Whether to check if all filenames have the same length.
                Default is False.
            verbose (bool): Whether to enable detailed logging.
                Default is False.

        Raises:
            FilenameLengthMismatchError: If filenames have different lengths and length_check is True.
            FileNotFoundError: If the configuration file does not exist.
            yaml.YAMLError: If the configuration file is invalid.
        """
        self.filenames = filenames
        self.verbose = verbose

        # Configure logging
        logger.remove()  # Remove the default handler
        if self.verbose:
            logger.add(lambda msg: print(msg, end=""), level="DEBUG", colorize=True)
        else:
            logger.add(lambda msg: print(msg, end=""), level="ERROR", colorize=True)

        # Only load patterns from config if no patterns are directly injected
        if config_path is None and not hasattr(self, "patterns"):
            self.patterns = None  # Set patterns to None initially
        elif config_path is not None:
            self.patterns = self.load_patterns(config_path)

        # Precompile regex patterns if patterns are loaded or injected
        if self.patterns:
            self.compiled_patterns = self.compile_patterns(self.patterns)

        if length_check:
            self._check_filename_lengths()

    def load_patterns(self, config_path: str) -> Dict[str, List[str]]:
        """
        Load patterns from a YAML configuration file.

        Args:
            config_path (str): Path to the YAML configuration file.

        Returns:
            Dict[str, List[str]]: A dictionary with R1, R2, and ignore patterns.

        Raises:
            FileNotFoundError: If the configuration file does not exist.
            yaml.YAMLError: If the configuration file is invalid.
        """
        if not os.path.isfile(config_path):
            logger.error(f"Configuration file not found at {config_path}")
            raise FileNotFoundError(f"Configuration file not found at {config_path}")

        try:
            with open(config_path, "r") as file:
                config = yaml.safe_load(file)
            return config["patterns"]
        except yaml.YAMLError as e:
            logger.error(f"Error parsing YAML configuration: {e}")
            raise

    def compile_patterns(self, patterns: Dict[str, List[str]]) -> Dict[str, List[re.Pattern]]:
        """
        Compile regex patterns for R1, R2, and ignore categories.

        Args:
            patterns (Dict[str, List[str]]): Raw patterns from configuration.

        Returns:
            Dict[str, List[re.Pattern]]: Compiled regex patterns.
        """
        compiled = {
            "R1": [re.compile(f".*({pattern})([._-]).*") for pattern in patterns.get("r1", [])],
            "R2": [re.compile(f".*({pattern})([._-]).*") for pattern in patterns.get("r2", [])],
            "ignore": [re.compile(f".*({pattern}).*") for pattern in patterns.get("ignore", [])],
        }
        return compiled

    def _check_filename_lengths(self) -> None:
        """
        Checks if all filenames have the same length.

        Raises:
            FilenameLengthMismatchError: If filenames do not have the same length.
        """
        lengths = list(map(len, self.filenames))
        if len(set(lengths)) > 1:
            logger.error("Filenames do not all have the same length. Please ensure all filenames are consistent.")
            raise FilenameLengthMismatchError("Filenames do not all have the same length. Please ensure all filenames are consistent.")

    def categorize_fastq_files(self) -> Dict[str, List[str]]:
        """
        Categorizes FASTQ files into R1, R2, or ignored based on filename patterns.

        Returns:
            Dict[str, List[str]]: A dictionary with keys 'R1', 'R2', and 'ignored',
                each containing lists of full file paths.

        Raises:
            PatternsNotLoadedError: If regex patterns are not loaded.
            FilenameLengthMismatchError: If the number of R1 and R2 files is unbalanced.
        """
        if not hasattr(self, "compiled_patterns"):
            raise PatternsNotLoadedError("Patterns not loaded or compiled.")

        categorized_files: Dict[str, List[str]] = {"R1": [], "R2": [], "ignored": []}

        for full_path in self.filenames:
            filename = os.path.basename(full_path)
            if any(pattern.search(filename) for pattern in self.compiled_patterns["ignore"]):
                categorized_files["ignored"].append(full_path)
            elif any(pattern.search(filename) for pattern in self.compiled_patterns["R1"]):
                categorized_files["R1"].append(full_path)
            elif any(pattern.search(filename) for pattern in self.compiled_patterns["R2"]):
                categorized_files["R2"].append(full_path)
            else:
                categorized_files["ignored"].append(full_path)

        # Sort the filenames alphabetically in each category
        for category in categorized_files:
            categorized_files[category].sort()

        # Check if the number of R1 and R2 files is balanced
        len_r1 = len(categorized_files.get("R1", []))
        len_r2 = len(categorized_files.get("R2", []))

        if len_r1 != len_r2:
            logger.error(f"Unbalanced categories: R1={len_r1}, R2={len_r2}")
            raise FilenameLengthMismatchError(f"Unbalanced categories: R1={len_r1}, R2={len_r2}")

        return categorized_files


Overwriting ../frmatcher/fastq_file_name_checker.py


In [95]:
# %load  ../frmatcher/fastq_file_name_checker.py
import os
import re
from typing import Dict, List, Optional

import yaml
from loguru import logger


class FilenameLengthMismatchError(ValueError):
    """Custom exception for filename length mismatches."""


class PatternsNotLoadedError(ValueError):
    """Custom exception when patterns are not loaded."""


class FastqFileNameChecker:
    def __init__(
        self,
        filenames: List[str],
        config_path: Optional[str] = None,
        length_check: bool = False,
        verbose: bool = False,
    ) -> None:
        """
        Initialize the FastqFileNameChecker with a list of filenames.

        Args:
            filenames (List[str]): List of file paths to categorize.
            config_path (str, optional): Path to the YAML configuration file.
                Defaults to None, and patterns must be manually injected.
            length_check (bool): Whether to check if all filenames have the same length.
                Default is False.
            verbose (bool): Whether to enable detailed logging.
                Default is False.

        Raises:
            FilenameLengthMismatchError: If filenames have different lengths and length_check is True.
            FileNotFoundError: If the configuration file does not exist.
            yaml.YAMLError: If the configuration file is invalid.
        """
        self.filenames = filenames
        self.verbose = verbose

        # Configure logging
        logger.remove()  # Remove the default handler
        if self.verbose:
            logger.add(lambda msg: print(msg, end=""), level="DEBUG", colorize=True)
        else:
            logger.add(lambda msg: print(msg, end=""), level="ERROR", colorize=True)

        # Only load patterns from config if no patterns are directly injected
        if config_path is None and not hasattr(self, "patterns"):
            self.patterns = None  # Set patterns to None initially
        elif config_path is not None:
            self.patterns = self.load_patterns(config_path)

        # Precompile regex patterns if patterns are loaded or injected
        if self.patterns:
            self.compiled_patterns = self.compile_patterns(self.patterns)

        if length_check:
            self._check_filename_lengths()

    def load_patterns(self, config_path: str) -> Dict[str, List[str]]:
        """
        Load patterns from a YAML configuration file.

        Args:
            config_path (str): Path to the YAML configuration file.

        Returns:
            Dict[str, List[str]]: A dictionary with R1, R2, and ignore patterns.

        Raises:
            FileNotFoundError: If the configuration file does not exist.
            yaml.YAMLError: If the configuration file is invalid.
        """
        if not os.path.isfile(config_path):
            logger.error(f"Configuration file not found at {config_path}")
            raise FileNotFoundError(f"Configuration file not found at {config_path}")

        try:
            with open(config_path, "r") as file:
                config = yaml.safe_load(file)
            return config["patterns"]
        except yaml.YAMLError as e:
            logger.error(f"Error parsing YAML configuration: {e}")
            raise

    def compile_patterns(
        self, patterns: Dict[str, List[str]]
    ) -> Dict[str, List[re.Pattern]]:
        """
        Compile regex patterns for R1, R2, and ignore categories.

        Args:
            patterns (Dict[str, List[str]]): Raw patterns from configuration.

        Returns:
            Dict[str, List[re.Pattern]]: Compiled regex patterns.
        """
        compiled = {
            "R1": [
                re.compile(f".*({pattern})([._-]).*")
                for pattern in patterns.get("r1", [])
            ],
            "R2": [
                re.compile(f".*({pattern})([._-]).*")
                for pattern in patterns.get("r2", [])
            ],
            "ignore": [
                re.compile(f".*({pattern}).*") for pattern in patterns.get("ignore", [])
            ],
        }
        return compiled

    def _check_filename_lengths(self) -> None:
        """
        Checks if all filenames have the same length.

        Raises:
            FilenameLengthMismatchError: If filenames do not have the same length.
        """
        lengths = list(map(len, self.filenames))
        if len(set(lengths)) > 1:
            logger.error(
                "Filenames do not all have the same length. Please ensure all filenames are consistent."
            )
            raise FilenameLengthMismatchError(
                "Filenames do not all have the same length. Please ensure all filenames are consistent."
            )

    def categorize_fastq_files(self) -> Dict[str, List[str]]:
        """
        Categorizes FASTQ files into R1, R2, or ignored based on filename patterns.

        Returns:
            Dict[str, List[str]]: A dictionary with keys 'R1', 'R2', and 'ignored',
                each containing lists of full file paths.

        Raises:
            PatternsNotLoadedError: If regex patterns are not loaded.
            FilenameLengthMismatchError: If the number of R1 and R2 files is unbalanced.
        """
        if not hasattr(self, "compiled_patterns"):
            raise PatternsNotLoadedError("Patterns not loaded or compiled.")

        categorized_files: Dict[str, List[str]] = {"R1": [], "R2": [], "ignored": []}

        for full_path in self.filenames:
            filename = os.path.basename(full_path)
            if any(
                pattern.search(filename) for pattern in self.compiled_patterns["ignore"]
            ):
                categorized_files["ignored"].append(full_path)
            elif any(
                pattern.search(filename) for pattern in self.compiled_patterns["R1"]
            ):
                categorized_files["R1"].append(full_path)
            elif any(
                pattern.search(filename) for pattern in self.compiled_patterns["R2"]
            ):
                categorized_files["R2"].append(full_path)
            else:
                categorized_files["ignored"].append(full_path)

        # Sort the filenames alphabetically in each category
        for category in categorized_files:
            categorized_files[category].sort()

        # Check if the number of R1 and R2 files is balanced
        len_r1 = len(categorized_files.get("R1", []))
        len_r2 = len(categorized_files.get("R2", []))

        if len_r1 != len_r2:
            logger.error(f"Unbalanced categories: R1={len_r1}, R2={len_r2}")
            raise FilenameLengthMismatchError(
                f"Unbalanced categories: R1={len_r1}, R2={len_r2}"
            )

        return categorized_files


In [96]:
# Sample usage
filenames = [
    "./_1/sample_R_L001.fastq.gz",  # R1 with lane info
    "./_1/sample_F_L001.fastq.gz",  # R2 with lane info
    "./_1/sample_R_L011.fastq.gz",  # R1 with lane info
    "./_1/sample_F_L011.fastq.gz",  # R2 with lane info
    # "sample_2_L011.fastq.gz",  # R2 with lane info
    "i_sample_1_L001.fastq.gz",  # Ignored due to prefix "i_"
    "I_sample_2_L001.fastq.gz",  # Ignored due to prefix "I_"
    #     "sample_r1_i1.fastq.gz",   # Ignored due to multiplex index
    "sample_A_L001.fastq.gz",  # R1 with lane info
    "sample_B_L001.fastq.gz",  # R2 with lane info
]

try:
    checker = FastqFileNameChecker(
        filenames,
        length_check=False,
        verbose=True,
        config_path="../frmatcher/config/patterns.yaml",
    )
    categorized_files = checker.categorize_fastq_files()

    # Output the categorized files
    for category, files in categorized_files.items():
        logger.info(f"{category}:")
        for file in files:
            logger.info(f"  - {file}")
except ValueError as e:
    logger.error(str(e))

categorized_files

2024-09-27 22:53:44.645 | INFO     | __main__:<module>:26 - R1:
2024-09-27 22:53:44.659 | INFO     | __main__:<module>:28 -   - ./_1/sample_F_L001.fastq.gz
2024-09-27 22:53:44.661 | INFO     | __main__:<module>:28 -   - ./_1/sample_F_L011.fastq.gz
2024-09-27 22:53:44.662 | INFO     | __main__:<module>:26 - R2:
2024-09-27 22:53:44.663 | INFO     | __main__:<module>:28 -   - ./_1/sample_R_L001.fastq.gz
2024-09-27 22:53:44.677 | INFO     | __main__:<module>:28 -   - ./_1/sample_R_L011.fastq.gz
2024-09-27 22:53:44.679 | INFO     | __main__:<module>:26 - ignored:
2024-09-27 22:53:44.680 | INFO     | __main__:<module>:28 -   - I_sample_2_L001.fastq.gz
2024-09-27 22:53:44.691 | INFO     | __main__:<module>:28 -   - i_sample_1_L001.fastq.gz
2024-09-27 22:53:44.692 | INFO     | __main__:<module>:28 -   - sample_A_L001.fastq.gz
2024-09-27 22:53:44.693 | INFO     | __main__:<module>:28 -   - sample_B_L001.fastq.gz


{'R1': ['./_1/sample_F_L001.fastq.gz', './_1/sample_F_L011.fastq.gz'],
 'R2': ['./_1/sample_R_L001.fastq.gz', './_1/sample_R_L011.fastq.gz'],
 'ignored': ['I_sample_2_L001.fastq.gz',
  'i_sample_1_L001.fastq.gz',
  'sample_A_L001.fastq.gz',
  'sample_B_L001.fastq.gz']}

In [97]:
! poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: frmatcher (0.2.0)Installing the current project: frmatcher (0.2.0)


In [98]:
from frmatcher import FastqFileNameChecker

filenames = [
    "sample_1_L001.fastq.gz",
    "sample_2_L001.fastq.gz",
    "sample_1_L002.fastq.gz",
    "sample_2_L002.fastq.gz",
]

checker = FastqFileNameChecker(filenames, length_check=False, verbose=False)

# checker = FastqFileNameChecker(filenames,
#                                length_check=True,
#                                verbose=True,
#                                config_path=None)
# checker.patterns = {
#     'r1': ["_1", "_R1"],
#     'r2': ["_2", "_R2"],
#     'ignore': ["^i_", "^I_", "_i\\d+", "_I\\d+"]
# }

categorized_files = checker.categorize_fastq_files()

print(categorized_files)

# {'R1': ['sample_1_L001.fastq.gz', 'sample_1_L002.fastq.gz'],
# 'R2': ['sample_2_L001.fastq.gz', 'sample_2_L002.fastq.gz'],
# 'ignored': []}

{'R1': ['sample_1_L001.fastq.gz', 'sample_1_L002.fastq.gz'], 'R2': ['sample_2_L001.fastq.gz', 'sample_2_L002.fastq.gz'], 'ignored': []}


In [94]:
from frmatcher import FastqFileNameChecker

filenames = [
    "sample_1_L001.fastq.gz",
    "sample_2_L001.fastq.gz",
    "sample_1_L002.fastq.gz",
    "sample_2_L002.fastq.gz",
]


checker = FastqFileNameChecker(
    filenames, length_check=True, verbose=False, config_path=None
)
checker.patterns = {
    "r1": ["_1", "_R1"],
    "r2": ["_2", "_R2"],
    "ignore": ["^i_", "^I_", "_i\\d+", "_I\\d+"],
}

categorized_files = checker.categorize_fastq_files()

print(categorized_files)

# {'R1': ['sample_1_L001.fastq.gz', 'sample_1_L002.fastq.gz'],
# 'R2': ['sample_2_L001.fastq.gz', 'sample_2_L002.fastq.gz'],
# 'ignored': []}

{'R1': ['sample_1_L001.fastq.gz', 'sample_1_L002.fastq.gz'], 'R2': ['sample_2_L001.fastq.gz', 'sample_2_L002.fastq.gz'], 'ignored': []}


In [87]:
%%writefile ../tests/test_fastq_file_name_checker.py
import unittest
import yaml
# Adjust the import according to your module structure
from frmatcher import FastqFileNameChecker

class TestFastqFileNameChecker(unittest.TestCase):

    def test_categorization(self):
        # Define the YAML patterns directly in the test
        patterns = yaml.safe_load(r"""
        patterns:
          r1:
            - "_1"
            - "_R1"
          r2:
            - "_2"
            - "_R2"
          ignore:
            - "^i_"
            - "^I_"
            - "_i\\d+"
            - "_I\\d+"
        """)

        filenames = [
            "sample_1_L001.fastq.gz",    # R1
            "sample_R1_L001.fastq.gz",   # R1
            "sample_2_L001.fastq.gz",    # R2
            "sample_R2_L001.fastq.gz",   # R2
            "sample_i1_L001.fastq.gz",   # Ignored
            "sample_I2_L001.fastq.gz",   # Ignored
            "i_sample_1_L001.fastq.gz",  # Ignored
            "I_sample_2_L001.fastq.gz",  # Ignored
            "sample_A_L001.fastq.gz"     # Ignored (no matching pattern)
        ]

        # Initialize checker without config_path, since we're injecting the patterns directly
        checker = FastqFileNameChecker(filenames)
        checker.patterns = patterns['patterns']  # Inject the test patterns directly
        checker.compiled_patterns = checker.compile_patterns(checker.patterns)  # Compile patterns after injection

        categorized_files = checker.categorize_fastq_files()

        # Assert correct categorization
        expected_r1 = {"sample_1_L001.fastq.gz", "sample_R1_L001.fastq.gz"}
        expected_r2 = {"sample_2_L001.fastq.gz", "sample_R2_L001.fastq.gz"}
        expected_ignored = {
            "sample_i1_L001.fastq.gz",
            "sample_I2_L001.fastq.gz",
            "i_sample_1_L001.fastq.gz",
            "I_sample_2_L001.fastq.gz",
            "sample_A_L001.fastq.gz"
        }

        with self.subTest("R1 Categorization"):
            self.assertEqual(set(categorized_files['R1']), expected_r1)

        with self.subTest("R2 Categorization"):
            self.assertEqual(set(categorized_files['R2']), expected_r2)

        with self.subTest("Ignored Categorization"):
            self.assertEqual(set(categorized_files['ignored']), expected_ignored)

if __name__ == '__main__':
    unittest.main()



Overwriting ../tests/test_fastq_file_name_checker.py


In [85]:
%%writefile ../.gitignore
# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class

# C extensions
*.so

# Distribution / packaging
.Python
env/
venv/
ENV/
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg
MANIFEST

# Installer logs
pip-log.txt
pip-delete-this-directory.txt

# Unit test / coverage reports
htmlcov/
.tox/
.nox/
coverage.xml
*.cover
*.py,cover
.hypothesis/
.pytest_cache/
cover/

# mypy
.mypy_cache/
.dmypy.json
dmypy.json

# Pyre type checker
.pyre/

# Environments
.env
.envrc
venv/
.venv/

# Poetry-specific files
poetry.lock
# If you use pyproject.toml for other purposes, uncomment the line below
# !pyproject.toml

# VS Code settings
.vscode/

# MacOS files
.DS_Store

# PyCharm settings
.idea/

# Local config files
*.local
*.log


Overwriting ../.gitignore


In [86]:
%%writefile ../LICENSE
MIT License

Copyright (c) 2024 Denis Odinokov

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

Overwriting ../LICENSE


In [10]:
from frmatcher.fastq_file_name_checker import FastqFileNameChecker

filenames = [
    "sample_1_L001.fastq.gz",
    "sample_2_L001.fastq.gz",
    "sample_1_L002.fastq.gz",
    "sample_2_L002.fastq.gz",
]

checker = FastqFileNameChecker(
    filenames, length_check=False, verbose=False, config_path=None
)
checker.patterns = {
    "r1": ["_1", "_R1"],
    "r2": ["_2", "_R2"],
    "ignore": ["^i_", "^I_", "_i\\d+", "_I\\d+"],
}
categorized_files = checker.categorize_fastq_files()

print(categorized_files)
# {'R1': ['sample_1_L001.fastq.gz', 'sample_1_L002.fastq.gz'], 'R2': ['sample_2_L001.fastq.gz', 'sample_2_L002.fastq.gz'], 'ignored': []}

{'R1': ['sample_1_L001.fastq.gz', 'sample_1_L002.fastq.gz'], 'R2': ['sample_2_L001.fastq.gz', 'sample_2_L002.fastq.gz'], 'ignored': []}


In [11]:
%%writefile ../README.md
# FRMatcher

**FRMatcher** categorizes a list of presumably FASTQ files into `R1` (forward reads) and `R2` (reverse reads) pairs using customizable pattern matching.

## Installation

Clone the repository:
   ```bash
   git clone https://github.com/odinokov/frmatcher.git
   cd frmatcher
   ```

Activate the virtual environment:
   ```bash
   poetry shell
   ```

Build the package:
   ```bash
   poetry build
   ```

Install the package locally:
   ```bash
   poetry install
   ```

## Usage

```python
from frmatcher.fastq_file_name_checker import FastqFileNameChecker

filenames = [
    "sample_1_L001.fastq.gz",
    "sample_2_L001.fastq.gz",
    "sample_1_L002.fastq.gz",
    "sample_2_L002.fastq.gz",
]

checker = FastqFileNameChecker(filenames,
                              length_check=False,
                              verbose=False)

# checker = FastqFileNameChecker(filenames,
#                                length_check=True,
#                                verbose=True,
#                                config_path=None)
# checker.patterns = {
#     'r1': ["_1", "_R1"],
#     'r2': ["_2", "_R2"],
#     'ignore': ["^i_", "^I_", "_i\\d+", "_I\\d+"]
# }

categorized_files = checker.categorize_fastq_files()

print(categorized_files)

# {'R1': ['sample_1_L001.fastq.gz', 'sample_1_L002.fastq.gz'], 
# 'R2': ['sample_2_L001.fastq.gz', 'sample_2_L002.fastq.gz'], 
# 'ignored': []}

```

## License

MIT License. See the [LICENSE](LICENSE) file for details.


Overwriting ../README.md
